In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/GAN_for_Neural_Graph/ADHD"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['ADHD-200_PhenotypicKey.pdf',
 'ADHD200_training_set_80_190x190.mat',
 'ADHD200_testing_set_20_190x190.mat',
 'ADHD_20%_Data_info_all.xlsx',
 'readme.txt',
 'ADHD_80%_Data_info_all.xlsx',
 'Data',
 'AC_Brain',
 'BrainNet',
 'compared_models',
 'BrainNetCNN',
 'Preprocessing.ipynb']

# Import labraries

In [2]:
import os
import xlrd
import keras
import scipy
import pandas as pd
from scipy import io
import numpy as np
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras import optimizers
from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import Sequential
from keras import backend
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Add
from keras.utils import conv_utils
from keras.utils import to_categorical
from keras.engine import Layer
from keras.engine import InputSpec
from keras.datasets.fashion_mnist import load_data
from keras.constraints import Constraint
from keras.initializers import RandomNormal
from keras.optimizers import Adam, RMSprop
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from matplotlib import pyplot

# Load data


In [3]:
# load data from xxx.mat
test_data = scipy.io.loadmat('ADHD200_testing_set_20_190x190.mat')
train_data = scipy.io.loadmat('ADHD200_training_set_80_190x190.mat')

In [4]:
# transpose: (190, 190, sample_size) ----> （sample_size, 190, 190）
test_control_data = test_data['sfc_ADHD200_adhd_20_190x190'].transpose(2, 0, 1)
test_adhd_data = test_data['sfc_ADHD200_controls_20_190x190'].transpose(2, 0, 1)
train_control_data = train_data['sfc_ADHD200_adhd_80_190x190'].transpose(2, 0, 1)
train_adhd_data = train_data['sfc_ADHD200_controls_80_190x190'].transpose(2, 0, 1)

# delete train_adhd_data's 15th data (wrong data)
train_adhd_data = np.delete(train_adhd_data, 14, axis=0)

test_data = np.concatenate((test_control_data, test_adhd_data), axis=0)
train_data = np.concatenate((train_control_data, train_adhd_data), axis=0)

print(test_data.shape, train_data.shape)

(171, 190, 190) (758, 190, 190)


In [ ]:
171+758

929

In [ ]:
a = len(test_data[np.isnan(test_data)])
b = len(train_data[np.isnan(train_data)])
print("test: ", a)
print("train: ", b)
print("total: ", (a + b)) 
  

test:  107730
train:  71820
total:  179550


# Matrix operation

In [ ]:
# matrix SVT
test_data = test_data.reshape(171, -1)
train_data = train_data.reshape(758, -1)

In [ ]:
!pip3 install fancyimpute

In [ ]:
from fancyimpute import SoftImpute

test_data_hat = SoftImpute().fit_transform(test_data)
train_data_hat = SoftImpute().fit_transform(train_data)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


[SoftImpute] Max Singular Value of X_init = 373.441278
[SoftImpute] Iter 1: observed MAE=0.031823 rank=169
[SoftImpute] Iter 2: observed MAE=0.031822 rank=169
[SoftImpute] Iter 3: observed MAE=0.031822 rank=169
[SoftImpute] Iter 4: observed MAE=0.031822 rank=169
[SoftImpute] Iter 5: observed MAE=0.031821 rank=169
[SoftImpute] Iter 6: observed MAE=0.031813 rank=168
[SoftImpute] Iter 7: observed MAE=0.031806 rank=168
[SoftImpute] Iter 8: observed MAE=0.031803 rank=168
[SoftImpute] Iter 9: observed MAE=0.031802 rank=168
[SoftImpute] Iter 10: observed MAE=0.031802 rank=168
[SoftImpute] Iter 11: observed MAE=0.031802 rank=168
[SoftImpute] Iter 12: observed MAE=0.031802 rank=168
[SoftImpute] Iter 13: observed MAE=0.031802 rank=168
[SoftImpute] Iter 14: observed MAE=0.031803 rank=168
[SoftImpute] Iter 15: observed MAE=0.031803 rank=168
[SoftImpute] Iter 16: observed MAE=0.031803 rank=168
[SoftImpute] Iter 17: observed MAE=0.031803 rank=168
[SoftImpute] Iter 18: observed MAE=0.031804 rank=168


In [ ]:
train_data = train_data_hat.reshape(-1, 190, 190)
test_data = test_data_hat.reshape(-1, 190, 190)

print(train_data.shape, test)

# Preprocessing

In [ ]:
train_data = train_data / 3
test_data = test_data / 3

In [ ]:
np.save('Data/test_data.npy', test_data)
np.save('Data/train_data.npy', train_data)

In [5]:
# load information from xxx.xlsx
def load_info(n, path):  # 0: Control, 1: ADHD
  xls = xlrd.open_workbook(path)
  table = xls.sheets()[n]
  nrows = table.nrows
  data = []

  for i in range(nrows):
    data.append(table.row_values(i))

  data = np.array(data)
  return data

In [6]:
# get information: gender age label
test_control_info = load_info(0, 'ADHD_20%_Data_info_all.xlsx')[1:, (2,3,5)]  
test_adhd_info = load_info(1, 'ADHD_20%_Data_info_all.xlsx')[1:, (2,3,5)]     
train_control_info = load_info(0, 'ADHD_80%_Data_info_all.xlsx')[1:, (2,3,5)]
train_adhd_info = load_info(1, 'ADHD_80%_Data_info_all.xlsx')[1:, (2,3,5)]

test_info = np.concatenate((test_control_info, test_adhd_info), axis=0).astype(float)
train_info = np.concatenate((train_control_info, train_adhd_info), axis=0).astype(float)

test_gender = test_info[:, 0].astype(np.int)
test_age = test_info[:, 1]
test_label = test_info[:, 2].astype(np.int)

train_gender = train_info[:, 0].astype(np.int)
train_age = train_info[:, 1]
train_label = train_info[:, 2].astype(np.int)

In [11]:
print(np.sum(train_label==0)+np.sum(test_label==0))
print(np.sum(train_label==1)+np.sum(test_label==1))
print(np.sum(train_label==2)+np.sum(test_label==2))
print(np.sum(train_label==3)+np.sum(test_label==3))

573
207
13
136


In [ ]:
# convert 2 to 1
# convert 3 to 2  
test_label[test_label == 2] = 1
test_label[test_label == 3] = 2
train_label[train_label == 2] = 1
train_label[train_label == 3] = 2

In [ ]:
# age normalization
# find the max age
max = np.max(np.array([np.max(test_age), np.max(train_age)]))  # 26.31
test_age = test_age / max
train_age = train_age / max

In [ ]:
train_combine = np.zeros((758, 3))
test_combine = np.zeros((171, 3))

train_combine[:, 0] = train_label
train_combine[:, 1] = train_age
train_combine[:, 2] = train_gender

test_combine[:, 0] = test_label
test_combine[:, 1] = test_age
test_combine[:, 2] = test_gender

print(train_combine[:10])

[[0.         0.35309768 0.        ]
 [0.         0.52261498 0.        ]
 [0.         0.42493349 0.        ]
 [0.         0.55036108 0.        ]
 [0.         0.45990118 1.        ]
 [0.         0.43025466 1.        ]
 [0.         0.28696313 0.        ]
 [0.         0.47662486 0.        ]
 [0.         0.51007222 1.        ]
 [0.         0.31394907 1.        ]]


In [ ]:
np.save('Data/train_combine', train_combine)
np.save('Data/test_combine', test_combine)

In [ ]:
# one-hot encoder
train_onehot = to_categorical(train_label)
test_onehot = to_categorical(test_label)

In [ ]:
# create encoded_data
train_encoded_data = np.zeros((758, 5))
test_encoded_data = np.zeros((171, 5))

train_encoded_data[:, :3] = train_onehot
train_encoded_data[:, 3:] = train_combine[:, 1:]

test_encoded_data[:, :3] = test_onehot
test_encoded_data[:, 3:] = test_combine[:, 1:]

print(train_encoded_data.shape, test_encoded_data.shape)
print(train_encoded_data[10])

(758, 5) (171, 5)
[1.         0.         0.         0.45115926 0.        ]
